# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jan 3 2023 11:52AM,254171,15,PNC398023A,"Person & Covey, Inc.",Released
1,Jan 3 2023 11:52AM,254171,15,PNC398032A,"Person & Covey, Inc.",Released
2,Jan 3 2023 11:52AM,254171,15,PNC398034A,"Person & Covey, Inc.",Released
3,Jan 3 2023 11:52AM,254171,15,PNC398060A,"Person & Covey, Inc.",Released
4,Jan 3 2023 11:52AM,254171,15,PNC398067A,"Person & Covey, Inc.",Released
5,Jan 3 2023 11:52AM,254171,15,PNC398068A,"Person & Covey, Inc.",Released
6,Jan 3 2023 11:52AM,254171,15,PNC398069A,"Person & Covey, Inc.",Released
7,Jan 3 2023 11:52AM,254171,15,PNC398070A,"Person & Covey, Inc.",Released
8,Jan 3 2023 11:52AM,254171,15,PNC380938C,"Person & Covey, Inc.",Released
9,Jan 3 2023 11:52AM,254171,15,PNC380944B,"Person & Covey, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
52,254167,Released,1
53,254169,Released,1
54,254170,Released,1
55,254171,Released,112
56,254172,Released,3


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
254167,NaN,NaN,1.0
254169,NaN,NaN,1.0
254170,NaN,NaN,1.0
254171,NaN,NaN,112.0
254172,NaN,NaN,3.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
254090,37.0,19.0,6.0
254104,0.0,0.0,1.0
254108,0.0,0.0,24.0
254111,0.0,0.0,53.0
254113,0.0,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
254090,37,19,6
254104,0,0,1
254108,0,0,24
254111,0,0,53
254113,0,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,254090,37,19,6
1,254104,0,0,1
2,254108,0,0,24
3,254111,0,0,53
4,254113,0,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,254090,37,19,6
1,254104,,,1
2,254108,,,24
3,254111,,,53
4,254113,,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jan 3 2023 11:52AM,254171,15,"Person & Covey, Inc."
112,Jan 3 2023 11:51AM,254172,19,VITABIOTICS USA INC.
115,Jan 3 2023 11:51AM,254157,15,"Alliance Pharma, Inc."
117,Jan 3 2023 11:46AM,254170,20,"Exact-Rx, Inc."
118,Jan 3 2023 11:45AM,254169,20,"Exact-Rx, Inc."
119,Jan 3 2023 11:43AM,254167,20,"Exact-Rx, Inc."
120,Jan 3 2023 11:42AM,254166,20,"Exact-Rx, Inc."
121,Jan 3 2023 11:40AM,254165,20,"Exact-Rx, Inc."
122,Jan 3 2023 11:21AM,254164,10,Emerginnova
123,Jan 3 2023 11:00AM,254163,12,"Uniderm USA, Inc"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jan 3 2023 11:52AM,254171,15,"Person & Covey, Inc.",,,112
1,Jan 3 2023 11:51AM,254172,19,VITABIOTICS USA INC.,,,3
2,Jan 3 2023 11:51AM,254157,15,"Alliance Pharma, Inc.",,,2
3,Jan 3 2023 11:46AM,254170,20,"Exact-Rx, Inc.",,,1
4,Jan 3 2023 11:45AM,254169,20,"Exact-Rx, Inc.",,,1
5,Jan 3 2023 11:43AM,254167,20,"Exact-Rx, Inc.",,,1
6,Jan 3 2023 11:42AM,254166,20,"Exact-Rx, Inc.",,,1
7,Jan 3 2023 11:40AM,254165,20,"Exact-Rx, Inc.",,,1
8,Jan 3 2023 11:21AM,254164,10,Emerginnova,,,1
9,Jan 3 2023 11:00AM,254163,12,"Uniderm USA, Inc",,,5


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 3 2023 11:52AM,254171,15,"Person & Covey, Inc.",112,,
1,Jan 3 2023 11:51AM,254172,19,VITABIOTICS USA INC.,3,,
2,Jan 3 2023 11:51AM,254157,15,"Alliance Pharma, Inc.",2,,
3,Jan 3 2023 11:46AM,254170,20,"Exact-Rx, Inc.",1,,
4,Jan 3 2023 11:45AM,254169,20,"Exact-Rx, Inc.",1,,
5,Jan 3 2023 11:43AM,254167,20,"Exact-Rx, Inc.",1,,
6,Jan 3 2023 11:42AM,254166,20,"Exact-Rx, Inc.",1,,
7,Jan 3 2023 11:40AM,254165,20,"Exact-Rx, Inc.",1,,
8,Jan 3 2023 11:21AM,254164,10,Emerginnova,1,,
9,Jan 3 2023 11:00AM,254163,12,"Uniderm USA, Inc",5,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 3 2023 11:52AM,254171,15,"Person & Covey, Inc.",112,,
1,Jan 3 2023 11:51AM,254172,19,VITABIOTICS USA INC.,3,,
2,Jan 3 2023 11:51AM,254157,15,"Alliance Pharma, Inc.",2,,
3,Jan 3 2023 11:46AM,254170,20,"Exact-Rx, Inc.",1,,
4,Jan 3 2023 11:45AM,254169,20,"Exact-Rx, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 3 2023 11:52AM,254171,15,"Person & Covey, Inc.",112.0,NaN,NaN
1,Jan 3 2023 11:51AM,254172,19,VITABIOTICS USA INC.,3.0,NaN,NaN
2,Jan 3 2023 11:51AM,254157,15,"Alliance Pharma, Inc.",2.0,NaN,NaN
3,Jan 3 2023 11:46AM,254170,20,"Exact-Rx, Inc.",1.0,NaN,NaN
4,Jan 3 2023 11:45AM,254169,20,"Exact-Rx, Inc.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 3 2023 11:52AM,254171,15,"Person & Covey, Inc.",112.0,0.0,0.0
1,Jan 3 2023 11:51AM,254172,19,VITABIOTICS USA INC.,3.0,0.0,0.0
2,Jan 3 2023 11:51AM,254157,15,"Alliance Pharma, Inc.",2.0,0.0,0.0
3,Jan 3 2023 11:46AM,254170,20,"Exact-Rx, Inc.",1.0,0.0,0.0
4,Jan 3 2023 11:45AM,254169,20,"Exact-Rx, Inc.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,7878303,835.0,40.0,0.0
12,1016525,6.0,2.0,0.0
15,1270709,135.0,60.0,0.0
19,1016534,12.0,20.0,37.0
20,1779090,7.0,0.0,0.0
21,254144,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,7878303,835.0,40.0,0.0
1,12,1016525,6.0,2.0,0.0
2,15,1270709,135.0,60.0,0.0
3,19,1016534,12.0,20.0,37.0
4,20,1779090,7.0,0.0,0.0
5,21,254144,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,835.0,40.0,0.0
1,12,6.0,2.0,0.0
2,15,135.0,60.0,0.0
3,19,12.0,20.0,37.0
4,20,7.0,0.0,0.0
5,21,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,835.0
1,12,Released,6.0
2,15,Released,135.0
3,19,Released,12.0
4,20,Released,7.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,19,20,21
Status,,,,,,
Completed,0.0,0.0,0.0,37.0,0.0,0.0
Executing,40.0,2.0,60.0,20.0,0.0,0.0
Released,835.0,6.0,135.0,12.0,7.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,19,20,21
0,Completed,0.0,0.0,0.0,37.0,0.0,0.0
1,Executing,40.0,2.0,60.0,20.0,0.0,0.0
2,Released,835.0,6.0,135.0,12.0,7.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,19,20,21
0,Completed,0.0,0.0,0.0,37.0,0.0,0.0
1,Executing,40.0,2.0,60.0,20.0,0.0,0.0
2,Released,835.0,6.0,135.0,12.0,7.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()